In [ ]:
cd ..

In [ ]:
ls

In [ ]:
import donkey_golf as dg

In [ ]:
import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

In [5]:
vamp = dg.data_things.LoadLeaderboard()

In [6]:
vamp.run()

2019-05-08 19:20:34,425 - donkey_golf.data_things.tourney_leaderboard - INFO - Tourney current status: finished
2019-05-08 19:20:34,426 - donkey_golf.data_things.tourney_leaderboard - INFO - Running clean_data function
2019-05-08 19:20:34,427 - donkey_golf.data_things.tourney_leaderboard - INFO - Running prepare_in_progress_data
2019-05-08 19:20:34,427 - donkey_golf.data_things.tourney_leaderboard - INFO - Calculating the donkey_score column
2019-05-08 19:20:34,446 - donkey_golf.data_things.tourney_leaderboard - INFO - Determining score for cut players and adding missed_cut indicator
2019-05-08 19:20:34,456 - donkey_golf.data_things.tourney_leaderboard - INFO - Determine if the player is active and if he has been cut or not
2019-05-08 19:20:34,474 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-08 19:20:34,772 - donkey_golf.data_things.tourney_leaderboard - DEBUG - Adding golfer team count
2019-05-08 19:20:34,783 - donkey_golf.data_things.tourney_leaderboard - I

In [7]:
vamp.tourney_status

'finished'

In [10]:
cmon = dg.data_things.PullLeaderboard(user_id=5)

2019-05-08 19:23:05,543 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-08 19:23:05,554 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-08 19:23:05,833 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056550


In [11]:
cmon.run()

2019-05-08 19:23:05,852 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-08 19:23:06,177 - donkey_golf.data_things.tourney_leaderboard - INFO - Tourney current status: finished
2019-05-08 19:23:06,179 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users active tourney team
2019-05-08 19:23:06,179 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-08 19:23:06,445 - donkey_golf.data_things.tourney_leaderboard - INFO - Determing data refresh times
2019-05-08 19:23:06,446 - donkey_golf.data_things.tourney_leaderboard - INFO - Tourney current status: finished
2019-05-08 19:23:06,447 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users active tourney team
2019-05-08 19:23:06,448 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [12]:
cmon.tourney_status

'finished'

In [ ]:
vamp.golfer_count_by_team

In [ ]:
df = vamp.data

In [ ]:
df.head(2)

In [ ]:
df.loc[df['team_count']> 0]

In [ ]:
huh = dg.data_things.PullLeaderboard(user_id=6)

In [ ]:
huh.run()

In [ ]:
huh.data['team_count'].value_counts()

In [ ]:
df_tourney = huh.data
df_tourney = df_tourney.loc[df_tourney['team_count'] > 0]

In [ ]:
from datetime import datetime

In [ ]:
datetime.today()

In [ ]:
df_tourney

In [ ]:
ada = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
ada.pull_tourney_leaderboard()

In [ ]:
ada.data.team_count.value_counts()

In [ ]:
ada.calc_refresh_date()

In [ ]:
ada.determine_tourney_status()

In [ ]:
ada.pull_users_team()

In [ ]:
ada.user_df

In [ ]:
ada.sql_user